# Simple Index Demo + ChatGPT

Use a very simple wrapper around the ChatGPT API

In [12]:
pip install llama-index

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Load documents, build the GPTSimpleVectorIndex

In [13]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import GPTSimpleVectorIndex, SimpleDirectoryReader, LLMPredictor
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display

In [15]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [17]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INPUT YOUR OPENAI_API_KEY"

index = GPTSimpleVectorIndex(documents, chunk_size_limit=512)

In [18]:
# LLM Predictor (gpt-3.5-turbo)
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))

#### Query Index

By default, with the help of langchain's PromptSelector abstraction, we use 
a modified refine prompt tailored for ChatGPT-use if the ChatGPT model is used.

In [19]:
response = index.query(
    "What did the author do growing up?", 
    llm_predictor=llm_predictor,
    similarity_top_k=3
)

In [20]:
display(Markdown(f"<b>{response}</b>"))

<b>Before college, the author worked on writing essays and programming. They wrote short stories and essays on various topics they had stacked up. However, in late 2015, the author stopped writing essays to focus on working on Bel, an interpreter written in itself. They worked on it intensively, often having a decent chunk of the code in their head at any given time. The author also worked on Bel while living in England, where they moved with their family in the summer of 2016. In the fall of 2019, Bel was finally finished, and the author resumed writing essays.</b>

In [21]:
response = index.query(
    "What did the author do during his time at RISD?", 
    llm_predictor=llm_predictor,
    similarity_top_k=5
)

In [22]:
display(Markdown(f"<b>{response}</b>"))

<b>The author took foundation classes in fundamental subjects like drawing, color, and design at RISD while in a PhD program in computer science at Harvard. He also painted during his free time and imagined himself living frugally off the royalties from a popular Lisp book he was writing and spending all his time painting. Additionally, he became a de facto studio assistant for a painter named Idelle Weber in New York. After learning a lot in the color class he took at RISD, he dropped out in 1993 and moved to New York to become a painter. He lived in a rent-controlled apartment in Yorkville and worked as a freelance Lisp hacker to support himself.</b>

**Refine Prompt**: Here is the chat refine prompt 

In [23]:
from llama_index.prompts.chat_prompts import CHAT_REFINE_PROMPT

In [24]:
dict(CHAT_REFINE_PROMPT.prompt)

{'input_variables': ['context_msg', 'existing_answer', 'query_str'],
 'output_parser': None,
 'partial_variables': {},
 'messages': [HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query_str'], output_parser=None, partial_variables={}, template='{query_str}', template_format='f-string', validate_template=True), additional_kwargs={}),
  AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['existing_answer'], output_parser=None, partial_variables={}, template='{existing_answer}', template_format='f-string', validate_template=True), additional_kwargs={}),
  HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context_msg'], output_parser=None, partial_variables={}, template="We have the opportunity to refine the above answer (only if needed) with some more context below.\n------------\n{context_msg}\n------------\nGiven the new context, refine the original answer to better answer the question. If the context isn't useful, output the original answ

#### Query Index (Using the standard Refine Prompt)

If we use the "standard" refine prompt (where the prompt is one text template instead of multiple messages), we find that the results over ChatGPT are worse. 

In [26]:
from llama_index.prompts.default_prompts import DEFAULT_REFINE_PROMPT

In [27]:
response = index.query(
    "What did the author do during his time at RISD?", 
    llm_predictor=llm_predictor,
    refine_template=DEFAULT_REFINE_PROMPT,
    similarity_top_k=5
)

In [28]:
display(Markdown(f"<b>{response}</b>"))

<b>The original answer is not sufficient to answer the question with the new context provided. There is no direct mention of what the author did during his time at RISD in the new context.</b>

### [Beta] Use ChatGPTLLMPredictor【Deprecated】

> The ChatGPTLLMPredictor is deprecated anyways, please check [issue/709](https://github.com/jerryjliu/llama_index/issues/709)

Very simple GPT-Index-native ChatGPT wrapper. Note: this is a beta feature. If this doesn't work please
use the suggested flow above.

In [29]:
# use ChatGPT [beta]
# from llama_index.langchain_helpers.chatgpt import ChatGPTLLMPredictor

# llm_predictor = ChatGPTLLMPredictor()

In [35]:
# response = index.query(
#     "What did the author do during his time at RISD?", 
#     llm_predictor=llm_predictor
# )

### Query Index (Using the ChatGPT Refine Prompt)

In [31]:
from llama_index.prompts.chat_prompts import CHAT_REFINE_PROMPT

In [33]:
# LLM Predictor (gpt-3.5-turbo)
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))

response = index.query(
    "What did the author do during his time at RISD?", 
    llm_predictor=llm_predictor,
    refine_template=CHAT_REFINE_PROMPT,
    similarity_top_k=5
)

In [34]:
display(Markdown(f"<b>{response}</b>"))

<b>The author attended RISD to learn how to paint and improve his skills. He took a color class and taught himself to paint, but ultimately dropped out in 1993. He then moved to New York City to pursue his career as an artist and lived in a rent-controlled apartment in Yorkville.</b>